In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd


data = pd.read_csv('data/EPC_Catalonia.csv')

/var/folders/xz/5bn1vdj146n6586g9r465k_40000gn/T/ipykernel_270/4013270058.py:9: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/EPC_Catalonia.csv')


In [2]:
df = data.copy()
# Define target variable (Energy Consumption)
target = "Consum d'energia final"

numeric_features = [
    "METRES_CADASTRE", "Energia primària no renovable", "Emissions de CO2",
    "Energia calefacció", "Energia refrigeració", "Energia calefacció demanda", "VALOR AILLAMENTS",
    "Cost anual aproximat d'energia per habitatge", "VALOR FINESTRES"
]

# Categorical features (to encode)
categorical_features = [
    "POBLACIO",  "Qualificació de consum d'energia primaria no renovable",
    "VEHICLE ELECTRIC", "SISTEMA BIOMASSA", "REHABILITACIO_ENERGETICA"]

# Convert binary categorical features to numeric (Yes/No -> 1/0)
binary_features = ["VEHICLE ELECTRIC", 
                   "SISTEMA BIOMASSA",  
                   "REHABILITACIO_ENERGETICA"]

for col in binary_features:
    df[col] = df[col].str.lower().map({'si': 1, 'no': 0})   

df = df[numeric_features + categorical_features + [target]].dropna()

In [3]:
# Encode categorical features
label_encoders = {}
for cat in categorical_features:
    le = LabelEncoder()
    df[cat] = le.fit_transform(df[cat])
    label_encoders[cat] = le  # Save for later
df = df[numeric_features + categorical_features + [target]].dropna()

# Normalize numerical features
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

X = df.drop(columns=[target]).values
y = df[target].values.reshape(-1,1)

In [4]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split into train and test (80-20)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [8]:
class EnergyNN(nn.Module):
    def __init__(self, input_dim):
        super(EnergyNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.output(x)
        return x


# Initialize model
input_dim = X.shape[1]
model = EnergyNN(input_dim)

# Define loss & optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -------------- TRAINING LOOP --------------
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# -------------- EVALUATION --------------
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch)
        predictions.append(preds.numpy())
        actuals.append(y_batch.numpy())


Epoch [0/100], Loss: 5928.1077
Epoch [10/100], Loss: 4503.3063
Epoch [20/100], Loss: 4488.4422
Epoch [30/100], Loss: 4318.8866
Epoch [40/100], Loss: 4261.1888
Epoch [50/100], Loss: 3682.4868
Epoch [60/100], Loss: 3526.7702
Epoch [70/100], Loss: 3382.3066
Epoch [80/100], Loss: 3550.5671
Epoch [90/100], Loss: 3007.9548


In [9]:
import numpy as np
predictions = np.vstack(predictions)
actuals = np.vstack(actuals)

# Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print(f"\nNeural Network Metrics:")
print(f"MAE: {mean_absolute_error(actuals, predictions):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(actuals, predictions)):.2f}")
print(f"R² Score: {r2_score(actuals, predictions):.2f}")


Neural Network Metrics:
MAE: 60.51
RMSE: 106.18
R² Score: 0.47


In [12]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create dataset
dataset = TensorDataset(X_tensor, y_tensor)
data_load = DataLoader(dataset, batch_size=64, shuffle=True)

input_dim = X.shape[1]
model = EnergyNN(input_dim)

# Define loss & optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -------------- TRAINING LOOP --------------
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in data_load:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(data_load):.4f}")

model.eval()


Epoch [0/100], Loss: 7593.3335
Epoch [10/100], Loss: 6229.1300
Epoch [20/100], Loss: 5488.6887
Epoch [30/100], Loss: 4907.3712
Epoch [40/100], Loss: 4679.8902
Epoch [50/100], Loss: 5433.5291
Epoch [60/100], Loss: 3871.0614
Epoch [70/100], Loss: 4485.9624
Epoch [80/100], Loss: 4190.0930
Epoch [90/100], Loss: 4361.8739


EnergyNN(
  (fc1): Linear(in_features=14, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [13]:

import torch

# Define filenames
filepaths = {
    "neural_network": "models/nn_model.pth"
}



# ---- Save Neural Network (PyTorch) ----
torch.save(model.state_dict(), filepaths["neural_network"])


print("\n✅ All objects saved successfully!")



✅ All objects saved successfully!
